A class to find a path through a maze using Greedy Best-First Search
and provide visualizations of the solution and heuristics.

In [1]:
import heapq
import copy

class MazeSolver:
    """
    A class to find a path through a maze using Greedy Best-First Search
    and provide visualizations of the solution and heuristics.
    """

    def __init__(self, maze):
        """
        Initializes the MazeSolver with a given maze.

        Args:
            maze (list[list]): A 2D list representing the maze where 'A' is the start,
                               'B' is the goal, 1 is a wall, and 0 is an open path.
        """
        if not maze:
            raise ValueError("Input maze cannot be empty.")
        self.original_maze = copy.deepcopy(maze)
        self.processed_maze, self.start, self.goal = self._process_maze()
        if not self.start or not self.goal:
            raise ValueError("Maze must contain a start ('A') and a goal ('B').")

    def _process_maze(self):
        """
        Converts the maze to numeric values and identifies start and goal positions.
        'A' and 'B' are treated as walkable paths (0).

        Returns:
            tuple: A tuple containing the processed maze (list[list]),
                   the start coordinates (tuple), and the goal coordinates (tuple).
        """
        processed = []
        start_pos, goal_pos = None, None
        for r, row in enumerate(self.original_maze):
            new_row = []
            for c, cell in enumerate(row):
                if cell == 'A':
                    start_pos = (r, c)
                    new_row.append(0)
                elif cell == 'B':
                    goal_pos = (r, c)
                    new_row.append(0)
                else:
                    new_row.append(cell)
            processed.append(new_row)
        return processed, start_pos, goal_pos

    @staticmethod
    def _manhattan_heuristic(a, b):
        """
        Calculates the Manhattan distance between two points.

        Args:
            a (tuple): The first point (row, col).
            b (tuple): The second point (row, col).

        Returns:
            int: The Manhattan distance.
        """
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    def _get_neighbors(self, pos):
        """
        Gets the valid, walkable neighbors of a given position in the maze.

        Args:
            pos (tuple): The current position (row, col).

        Returns:
            list[tuple]: A list of neighbor coordinates.
        """
        neighbors = []
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right
        rows, cols = len(self.processed_maze), len(self.processed_maze[0])
        for dr, dc in directions:
            nr, nc = pos[0] + dr, pos[1] + dc
            if 0 <= nr < rows and 0 <= nc < cols and self.processed_maze[nr][nc] != 1:
                neighbors.append((nr, nc))
        return neighbors

    def solve_with_greedy_bfs(self):
        """
        Finds the path from 'A' to 'B' using Greedy Best-First Search algorithm.
        The search is guided by the Manhattan distance heuristic.

        Returns:
            list[tuple]: The path from start to goal as a list of coordinates,
                         or an empty list if no path is found.
        """
        visited = set()
        came_from = {}
        
        # Priority queue stores (heuristic_cost, position)
        priority_queue = []
        heapq.heappush(priority_queue, (self._manhattan_heuristic(self.start, self.goal), self.start))

        while priority_queue:
            _, current = heapq.heappop(priority_queue)

            if current == self.goal:
                break # Goal reached

            if current in visited:
                continue
            visited.add(current)

            for neighbor in self._get_neighbors(current):
                if neighbor not in visited:
                    priority = self._manhattan_heuristic(neighbor, self.goal)
                    heapq.heappush(priority_queue, (priority, neighbor))
                    # Check to avoid overwriting a path from a better parent
                    if neighbor not in came_from:
                         came_from[neighbor] = current
        
        # Reconstruct path from goal to start
        path = []
        if self.goal not in came_from and self.start != self.goal:
             print("No path found.")
             return []

        curr = self.goal
        while curr != self.start:
            path.append(curr)
            curr = came_from.get(curr)
            # If curr becomes None, it means the path is broken.
            if curr is None and self.start != self.goal:
                print("No path found.")
                return []
        path.append(self.start)
        path.reverse()
        return path

    def get_path_visualization(self, path):
        """
        Marks a given path on the maze for visualization.

        Args:
            path (list[tuple]): The path to visualize.

        Returns:
            list[list]: A new maze with the path marked by '*'.
        """
        maze_copy = copy.deepcopy(self.original_maze)
        for r, c in path:
            if (r, c) != self.start and (r, c) != self.goal:
                maze_copy[r][c] = '*'
        return maze_copy

    def get_manhattan_visualization(self):
        """
        Creates a grid showing the Manhattan distance from each cell to the goal.

        Returns:
            list[list]: A grid where each cell contains its Manhattan distance to the goal,
                        or a wall character '█'.
        """
        rows, cols = len(self.original_maze), len(self.original_maze[0])
        dist_matrix = []
        for i in range(rows):
            row = []
            for j in range(cols):
                if self.original_maze[i][j] == 1:
                    row.append("█")
                elif (i, j) == self.start:
                    row.append('A')
                elif (i, j) == self.goal:
                    row.append('B')
                else:
                    row.append(self._manhattan_heuristic((i, j), self.goal))
            dist_matrix.append(row)
        return dist_matrix

    @staticmethod
    def print_matrix(matrix):
        """
        Prints a matrix (maze, path, or heuristic) in a readable format.

        Args:
            matrix (list[list]): The matrix to print.
        """
        for row in matrix:
            row_str = ""
            for val in row:
                if isinstance(val, str):
                    row_str += f" {val} "
                else:
                    row_str += f"{val:2d} "  # Align numbers with 2 spaces
            print(row_str)
        print() # Adds a newline for better spacing


if __name__ == '__main__':
    # Define the maze to be solved
    maze5 = [
        ['A', 0,  1,  0,  0,  0, 1, 0, 0, 1, 0, 0],
        [0,   0,  1,  0,  1,  0, 1, 0, 1, 0, 0, 0],
        [1,   0,  0,  0,  1,  0, 0, 0, 1, 1, 1, 0],
        [0,   1,  1,  0,  0,  1, 1, 0, 0, 0, 1, 0],
        [0,   0,  0,  0,  1,  0, 1, 1, 1, 0, 0, 0],
        [1,   1,  1,  0,  1,  0, 0, 0, 1, 1, 1, 0],
        [0,   0,  1,  0,  0,  0, 1, 0, 0, 0, 1, 0],
        [0,   1,  0,  1,  1,  0, 1, 1, 1, 0, 1, 0],
        [0,   0,  0,  0,  0,  0, 0, 0, 1, 0, 0, 0],
        [1,   1,  1,  1,  1,  1, 0, 1, 0, 1, 1, 0],
        [0,   0,  0,  0,  0,  0, 0, 1, 0, 0, 0, 0],
        [0,   1,  1,  1,  1,  1, 1, 1, 1, 1, 1,'B'],
    ]

    # 1. Create a solver instance for the maze
    solver = MazeSolver(maze5)

    # 2. Solve the maze and get the path
    print("Solving the maze using Greedy Best-First Search...")
    path_solution = solver.solve_with_greedy_bfs()

    if path_solution:
        print("\nPath found:")
        print(path_solution)

        # 3. Visualize the path on the maze
        print("\nVisualized Path on Maze:")
        visualized_path_maze = solver.get_path_visualization(path_solution)
        MazeSolver.print_matrix(visualized_path_maze)

    # 4. Visualize the Manhattan distance heuristic
    print("\nManhattan Distance Heuristic to Goal 'B':")
    manhattan_grid = solver.get_manhattan_visualization()
    MazeSolver.print_matrix(manhattan_grid)


Solving the maze using Greedy Best-First Search...

Path found:
[(0, 0), (0, 1), (1, 1), (2, 1), (2, 2), (2, 3), (3, 3), (4, 3), (5, 3), (6, 3), (6, 4), (6, 5), (5, 5), (5, 6), (5, 7), (6, 7), (6, 8), (6, 9), (7, 9), (8, 9), (8, 10), (8, 11), (9, 11), (10, 11), (11, 11)]

Visualized Path on Maze:
 A  *  1  0  0  0  1  0  0  1  0  0 
 0  *  1  0  1  0  1  0  1  0  0  0 
 1  *  *  *  1  0  0  0  1  1  1  0 
 0  1  1  *  0  1  1  0  0  0  1  0 
 0  0  0  *  1  0  1  1  1  0  0  0 
 1  1  1  *  1  *  *  *  1  1  1  0 
 0  0  1  *  *  *  1  *  *  *  1  0 
 0  1  0  1  1  0  1  1  1  *  1  0 
 0  0  0  0  0  0  0  0  1  *  *  * 
 1  1  1  1  1  1  0  1  0  1  1  * 
 0  0  0  0  0  0  0  1  0  0  0  * 
 0  1  1  1  1  1  1  1  1  1  1  B 


Manhattan Distance Heuristic to Goal 'B':
 A 21  █ 19 18 17  █ 15 14  █ 12 11 
21 20  █ 18  █ 16  █ 14  █ 12 11 10 
 █ 19 18 17  █ 15 14 13  █  █  █  9 
19  █  █ 16 15  █  █ 12 11 10  █  8 
18 17 16 15  █ 13  █  █  █  9  8  7 
 █  █  █ 14  █ 12 11 10  █  █